In [1]:
# import os
# import shutil

original_dataset_dir = './dataset'
# dataset안에 있는 하위 모든 폴더의 목록을 가져온다
classes_list = os.listdir(original_dataset_dir)

# 기본 경로
base_dir = './splitted'
os.mkdir(base_dir)

# train, val, test 용도의 사진을 저장하기 위해 폴더 생성
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'val')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# 하위 폴더 목록을 동일하게 생성
for clss in classes_list:
    os.mkdir(os.path.join(train_dir, clss))
    os.mkdir(os.path.join(validation_dir, clss))    
    os.mkdir(os.path.join(test_dir, clss))    

In [2]:
# import math
# 모든 하위 폴더에서 동작
for clss in classes_list:
    path = os.path.join(original_dataset_dir, clss)
    # 경로에 있는 모든 이미지 파일의 목록을 받아온다
    fnames = os.listdir(path)
    
    # train, val, test 용도의 비율을 조절한다
    # 6:2:2 사용
    train_size = math.floor(len(fnames)*0.6)
    validation_size = math.floor(len(fnames)*0.2)
    test_size = math.floor(len(fnames)*0.2)
    
    # train 데이터 만큼을 train에 저장
    train_fnames = fnames[:train_size]
    print('Train size(',clss,'): ',len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(train_dir, clss), fname)
        shutil.copyfile(src,dst)
        
    # val 데이터 만큼을 val에 저장
    validation_fnames = fnames[train_size:(validation_size + train_size)]
    print('Validation size(',clss,'): ',len(validation_fnames))
    for fname in validation_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir, clss), fname)
        shutil.copyfile(src,dst)
    
    # test 데이터 만큼을 test에 저장
    test_fnames = fnames[(validation_size + train_size):(test_size + validation_size + train_size)]
    print('Test size(',clss,'): ',len( test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, clss), fname)
        shutil.copyfile(src,dst)

Train size( Apple___Apple_scab ):  600
Validation size( Apple___Apple_scab ):  200
Test size( Apple___Apple_scab ):  200
Train size( Apple___Black_rot ):  600
Validation size( Apple___Black_rot ):  200
Test size( Apple___Black_rot ):  200
Train size( Apple___Cedar_apple_rust ):  600
Validation size( Apple___Cedar_apple_rust ):  200
Test size( Apple___Cedar_apple_rust ):  200
Train size( Apple___healthy ):  987
Validation size( Apple___healthy ):  329
Test size( Apple___healthy ):  329
Train size( Background_without_leaves ):  685
Validation size( Background_without_leaves ):  228
Test size( Background_without_leaves ):  228
Train size( Blueberry___healthy ):  901
Validation size( Blueberry___healthy ):  300
Test size( Blueberry___healthy ):  300
Train size( Cherry___healthy ):  600
Validation size( Cherry___healthy ):  200
Test size( Cherry___healthy ):  200
Train size( Cherry___Powdery_mildew ):  631
Validation size( Cherry___Powdery_mildew ):  210
Test size( Cherry___Powdery_mildew )

In [2]:
import os
import shutil
import torch
import math

# GPU 사용 체크
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else'cpu')
print('Current cuda device is', DEVICE)

# batch_size, epoch 설정 
BATCH_SIZE = 256
EPOCH = 30

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

# compose는 이미지 데이터의 전처리, Augmentaion에 사용
# Augmentaion의 경우 좌우반전, 밝기 조절, 이미지 임의 확대 등
# 이미지는 64*64 로 설정
transform_base = transforms.Compose([transforms.Resize((64,64)),
                                     transforms.ToTensor()])

# ImageFolder는 dataset을 불러오는 메서드 
train_dataset = ImageFolder(root='./splitted/train',
                            transform = transform_base)
val_dataset = ImageFolder(root='./splitted/val',
                            transform = transform_base)

from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size = BATCH_SIZE, shuffle = True,
                                           num_workers = 4)
val_loader = torch.utils.data.DataLoader(val_dataset,
                                         batch_size = BATCH_SIZE, shuffle = True, num_workers = 4)

Current cuda device is cuda


In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    
    # 모델 레이어 설정
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        
        self.fc1 = nn.Linear(4096, 512)
        self.fc2 = nn.Linear(512, 39)
        
    # forward 설정
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = x.view(-1, 4096)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)
    
# 모델 객체 생성 및 optimizer 설정
model_base = Net().to(DEVICE)
optimizer = optim.Adam(model_base.parameters(), lr=0.001)

In [4]:
print(model_base)

Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=39, bias=True)
)


In [5]:
# 모델 학습
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [6]:
def evaluate(model, test_loader):
    model.eval()
    # 미니 배치별 loss 합산
    test_loss = 0
    # 맞은 갯수 
    correct = 0
    
    # 파라미터 업데이트 중단
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            # loss 계산 cross entropy 사용
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()
            
            # 예측값 추출
            pred = output.max(1, keepdim=True)[1]
            # view as 로 Tensor 재정렬 후 비교연산 시행
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    # 미니 배치당 평균 loss 
    test_loss /= len(test_loader.dataset)
    # 미니 배치당 평균 정확도
    test_accuracy = 100.0 * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [7]:
import time
import copy

torch.backends.cudnn.enabled = False
def train_baseline(model, train_loader, val_loaderm,
                   optimizer, num_epochs = 30):
    # 최고 정확도 저장을 위한 변수
    best_acc = 0.0
    # 최고 정확도 모델을 저장하기 위한 변수
    best_model_wts = copy.deepcopy(model.state_dict())
    
    for epoch in range(1, num_epochs + 1):
        since = time.time()
        train(model, train_loader, optimizer)
        # 학습 Loss와 정확도
        train_loss, train_acc = evaluate(model, train_loader)
        # 검증 Loss와 정확도
        val_loss, val_acc = evaluate(model, val_loader)
        # 가장 좋은 경우에 최고 정확도, 모델 갱신
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            
        time_elapsed = time.time() - since
        print('-------------- epoch {} --------------'.format(epoch))
        print('train Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))
        print('val Loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    # 최고 정확도 모델을 불러오고 이를 return
    model.load_state_dict(best_model_wts)
    return model

base = train_baseline(model_base, train_loader, val_loader, optimizer, EPOCH)
# 학습완료 모델 저장
torch.save(base, 'baseline.pt')

-------------- epoch 1 --------------
train Loss: 1.4852, Accuracy: 58.96%
val Loss: 1.5399, Accuracy: 57.12%
Completed in 0m 48s
-------------- epoch 2 --------------
train Loss: 0.9777, Accuracy: 70.60%
val Loss: 1.0554, Accuracy: 68.27%
Completed in 0m 45s
-------------- epoch 3 --------------
train Loss: 0.6795, Accuracy: 79.45%
val Loss: 0.7791, Accuracy: 76.61%
Completed in 0m 47s
-------------- epoch 4 --------------
train Loss: 0.4923, Accuracy: 85.20%
val Loss: 0.6093, Accuracy: 81.88%
Completed in 0m 47s
-------------- epoch 5 --------------
train Loss: 0.4667, Accuracy: 85.97%
val Loss: 0.5771, Accuracy: 82.71%
Completed in 0m 47s
-------------- epoch 6 --------------
train Loss: 0.4242, Accuracy: 86.79%
val Loss: 0.5668, Accuracy: 82.95%
Completed in 0m 47s
-------------- epoch 7 --------------
train Loss: 0.4335, Accuracy: 86.34%
val Loss: 0.5807, Accuracy: 82.46%
Completed in 0m 47s
-------------- epoch 8 --------------
train Loss: 0.3051, Accuracy: 90.51%
val Loss: 0.448

In [9]:
# Transfer Learning 사용
# 데이터 정리
data_transforms = {
    'train': transforms.Compose([ # 이미지 전처리를 위한 Compose
        transforms.Resize([64, 64]),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomCrop(52), # 랜덤 위치의 52*52 사이즈 자르기
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], # 정규화 하기 / 평균 값과 표준편차 값이 필요
                             [0.229, 0.224, 0.225]) # 첫 대괄호 안이 RGB 평균 / 두 번째 대괄호가 각 표준편차
                                                    # 여기서 사용된 값들은 ImageNet 데이터의 값이다.
    ]),
    'val': transforms.Compose([
        transforms.Resize([64, 64]),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

data_dir = './splitted'
image_datasets = {x: ImageFolder(root= os.path.join(data_dir, x),
                                 transform= data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                 batch_size= BATCH_SIZE, shuffle= True, num_workers= 4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [10]:
# Pre-Trained Model 불러오기
from torchvision import models

resnet = models.resnet50(pretrained=True)
num_ftrs = resnet.fc.in_features # 마지막 레이어의 입력 채널 수
resnet.fc = nn.Linear(num_ftrs, 39) # 새로운 레이어로 교체 / 입력 채널 수는 동일, 출력은 39
resnet = resnet.to(DEVICE) # 장비에 할당

criterion = nn.CrossEntropyLoss()

# 모든 파라미터를 다 하는 것이 아닌 requires_grad 가 True 인 부분만 업데이트
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr= 0.001)

from torch.optim import lr_scheduler

# learning rate 를 epoch에 따라서 줄여주는 메서드 / 여기서는 7 epoch 마다 0.1을 lr에 곱해준다.
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size= 7, gamma= 0.1)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\gch05/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
24.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

80.4%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
# Layer Freeze
ct = 0
for child in resnet.children(): # 자식 모듈을 객체로 변환해서 넘긴다 / 모든 레이어를 가지고 온다
    ct += 1
    if ct < 6: # 6번 미만의 레이어는 업데이트를 꺼준다
        for param in child.parameters():
            param.requires_grad = False

In [19]:
def train_resnet(model, criterion, optimizer, scheduler, num_epochs=25):
    
    # 가장 높은 정확도의 모델 저장 및 정확도 저장
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('-------------- epoch {} --------------'.format(epoch + 1))
        since = time.time()
        
        # 한 epoch 마다 train과 val을 한 번씩 진행한다
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            # 모든 데이터의 loss를 합산해서 저장
            running_loss = 0.0
            # 맞게 예측한 경우의 수를 세서 저장
            running_corrects = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)
                
                optimizer.zero_grad()
                
                # 학습의 경우에만 gradient 업데이트가 이루어지게 설정
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    # 각 클레스에 포함될 확률 중 가장 큰 인덱스를 가져온다
                    _, preds = torch.max(outputs, 1)
                    # 예측 값과 target 사이의 오차를 계산한다
                    loss = criterion(outputs, labels)
                    
                    # train 일 경우에만 backpropagation을 진행한다.
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                # 하나의 미니 베치에서 loss를 데이터 수 만큼 곱해서 더해준다
                running_loss += loss.item() * inputs.size(0)
                # 예측 값과 target이 같으면 1을 증가시켜준다
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':
                # 학습률 조정을 위한 scheduler 사용
                scheduler.step()
                # 각 epoch의 l_r 을 표현한다
                l_r = [x['lr'] for x in optimizer_ft.param_groups]
                print('learning rate: ', l_r)
                
            # epoch_loss 와 epoch_acc 를 위해서 running_lose 및 running_corrects 를 데이터 셋 사이즈로 나눈다
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            # 좀 더 정확한 모델의 경우 업데이트
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
        time_elapsed = time.time() - since
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        
    print('Best val ACC: {:4f}'.format(best_acc))
    
    model.load_state_dict(best_model_wts)
    
    return model

In [20]:
model_resnet50 = train_resnet(resnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs = EPOCH)

torch.save(model_resnet50, 'resnet50.pt')

-------------- epoch 1 --------------
learning rate:  [0.001]
train Loss: 0.1985 Acc: 0.9363
val Loss: 0.2131 Acc: 0.9392
Completed in 3m 39s
-------------- epoch 2 --------------
learning rate:  [0.001]
train Loss: 0.1541 Acc: 0.9511
val Loss: 0.1684 Acc: 0.9472
Completed in 3m 40s
-------------- epoch 3 --------------
learning rate:  [0.001]
train Loss: 0.1374 Acc: 0.9575
val Loss: 0.1205 Acc: 0.9599
Completed in 3m 39s
-------------- epoch 4 --------------
learning rate:  [0.001]
train Loss: 0.0949 Acc: 0.9687
val Loss: 0.1149 Acc: 0.9637
Completed in 3m 40s
-------------- epoch 5 --------------
learning rate:  [0.001]
train Loss: 0.0709 Acc: 0.9764
val Loss: 0.1576 Acc: 0.9593
Completed in 3m 39s
-------------- epoch 6 --------------
learning rate:  [0.0001]
train Loss: 0.0779 Acc: 0.9748
val Loss: 0.0987 Acc: 0.9690
Completed in 3m 39s
-------------- epoch 7 --------------
learning rate:  [0.0001]
train Loss: 0.0378 Acc: 0.9883
val Loss: 0.0367 Acc: 0.9880
Completed in 3m 39s
----

In [23]:
# 모델평가
# test data의 dataloader를 생성
# 학습, 검증과 동일한 방법으로 전처리를 해줘야 한다
transform_base = transforms.Compose([transforms.Resize([64, 64]),
                                     transforms.ToTensor()])
test_base = ImageFolder(root='./splitted/test',
                        transform= transform_base)
test_loader_base = torch.utils.data.DataLoader(test_base,
                                               batch_size= BATCH_SIZE, shuffle= True, num_workers= 4)

In [26]:
# 동일하게 transfer learning 모델의 데이터 전처리
transform_resNet = transforms.Compose([
    transforms.Resize([64, 64]),
    transforms.RandomCrop(52),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
test_resNet = ImageFolder(root='./splitted/test',
                        transform= transform_resNet)
test_loader_resNet = torch.utils.data.DataLoader(test_resNet,
                                                 batch_size= BATCH_SIZE, shuffle= True, num_workers= 4)

In [28]:
# 저장된 베이스라인 모델을 가져온다
# 이후 정확도 측정
baseline = torch.load('baseline.pt')
baseline.eval()
test_loss, test_accuracy = evaluate(baseline, test_loader_base)

print('baseline test acc: ', test_accuracy)

baseline test acc:  94.10855236390267


In [29]:
# 저장된 resnet50 모델을 가져온다
# 이후 정확도 측정
resnet50 = torch.load('resnet50.pt')
resnet50.eval()
test_loss, test_accuracy = evaluate(resnet50, test_loader_resNet)

print('ResNet test acc: ', test_accuracy)

ResNet test acc:  99.21067621450078


In [1]:
# Pre-Trained 모델의 경우가 다양한 이미지 Feature가 학습 되었기 때문에
# 이를 Fine-Tuning을 해준 resnet50.pt 버전이 정확도가 높게 나온 것을 볼 수 있다.